In [2]:
import numpy
import urllib
import scipy.optimize
import random

# Read Data

In [3]:
def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)

In [4]:
print "Reading data..."
data = list(parseData("file:beer_50000.json"))
print "done"

Reading data...
done


# Preprocess

In [5]:
style_num = {}
review_taste_total = {}
review_taste_average = {}

for single_data in data:
    if style_num.has_key(single_data['beer/style']):
        style_num[single_data['beer/style']] += 1
        review_taste_total[single_data['beer/style']] += single_data['review/taste']
    else:
        style_num[single_data['beer/style']] = 1
        review_taste_total[single_data['beer/style']] = single_data['review/taste']
# style_num stores the answer for tasks-regression problem 1
print style_num

{'Tripel': 257, 'American Black Ale': 138, 'Fruit / Vegetable Beer': 1355, 'English Brown Ale': 495, 'American Pale Ale (APA)': 2288, 'American Amber / Red Lager': 42, 'Chile Beer': 11, 'Cream Ale': 69, 'Dubbel': 165, 'English Dark Mild Ale': 21, 'Belgian Strong Pale Ale': 632, 'Weizenbock': 13, 'Milk / Sweet Stout': 69, 'Dunkelweizen': 61, 'Belgian Strong Dark Ale': 146, 'Foreign / Export Stout': 55, 'Low Alcohol Beer': 7, 'Euro Pale Lager': 701, 'Schwarzbier': 53, 'Munich Dunkel Lager': 141, 'Maibock / Helles Bock': 225, 'Belgian Pale Ale': 144, 'Irish Dry Stout': 101, 'M\xc3\xa4rzen / Oktoberfest': 557, 'California Common / Steam Beer': 11, 'American Porter': 2230, 'Vienna Lager': 33, 'Oatmeal Stout': 102, 'Bi\xc3\xa8re de Garde': 7, 'Doppelbock': 873, 'American Adjunct Lager': 242, 'Wheatwine': 455, 'Scottish Ale': 78, 'English Porter': 367, 'Saison / Farmhouse Ale': 141, 'Altbier': 165, 'English India Pale Ale (IPA)': 175, 'Dortmunder / Export Lager': 31, 'English Bitter': 267, 'S

In [6]:
for style in review_taste_total.keys():
    review_taste_average[style] = review_taste_total[style] / style_num[style]
# review_taste_average stores the answer for tasks-regression problem 1   
print review_taste_average

{'Tripel': 3.7840466926070038, 'American Black Ale': 3.8731884057971016, 'Fruit / Vegetable Beer': 3.607749077490775, 'English Brown Ale': 3.728282828282828, 'American Pale Ale (APA)': 3.649694055944056, 'American Amber / Red Lager': 3.6904761904761907, 'Chile Beer': 3.9545454545454546, 'Cream Ale': 3.028985507246377, 'Dubbel': 3.7363636363636363, 'English Dark Mild Ale': 3.7857142857142856, 'Belgian Strong Pale Ale': 4.056170886075949, 'Weizenbock': 3.3846153846153846, 'Milk / Sweet Stout': 3.782608695652174, 'Dunkelweizen': 3.4918032786885247, 'Belgian Strong Dark Ale': 3.6952054794520546, 'Foreign / Export Stout': 3.2545454545454544, 'American Porter': 4.081838565022421, 'American Double / Imperial Pilsner': 3.8214285714285716, 'Euro Pale Lager': 2.962910128388017, 'Pumpkin Ale': 3.7875, 'Munich Dunkel Lager': 3.780141843971631, 'Maibock / Helles Bock': 3.7466666666666666, 'Belgian Pale Ale': 3.7395833333333335, 'Irish Dry Stout': 3.623762376237624, 'M\xc3\xa4rzen / Oktoberfest': 3.

# Feature

In [7]:
def feature(datum):
  if datum['beer/style'] == 'American IPA':
    feat = [1,1]
  else:
    feat = [1,0]
  return feat

X = [feature(d) for d in data]
y = [d['review/taste'] for d in data]
theta,residuals,rank,s = numpy.linalg.lstsq(X, y)

# Loss Function

In [11]:
### Gradient descent ###
# Objective
def f(theta, X, y, lam):
  theta = numpy.matrix(theta).T
  X = numpy.matrix(X)
  y = numpy.matrix(y).T
  diff = X*theta - y
  diffSq = diff.T*diff
  diffSqReg = diffSq / len(X) + lam*(theta.T*theta)
  print "offset =", diffSqReg.flatten().tolist()
  return diffSqReg.flatten().tolist()[0]
# Derivative
def fprime(theta, X, y, lam):
  theta = numpy.matrix(theta).T
  X = numpy.matrix(X)
  y = numpy.matrix(y).T
  diff = X*theta - y
  res = 2*X.T*diff / len(X) + 2*lam*theta
  print "gradient =", numpy.array(res.flatten().tolist()[0])
  return numpy.array(res.flatten().tolist()[0])

scipy.optimize.fmin_l_bfgs_b(f, [theta[0],theta[1]], fprime, args = (X, y, 0))

offset = [[0.5130161734617561]]
gradient = [ -1.08186669e-13  -1.64709490e-14]


(array([ 3.91520474,  0.08564622]),
 [0.5130161734617561],
 {'funcalls': 1,
  'grad': array([ -1.08186669e-13,  -1.64709490e-14]),
  'nit': 0,
  'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'warnflag': 0})

In [14]:
theta = scipy.optimize.fmin_l_bfgs_b(f, [theta[0],theta[1]], fprime, args = (X, y, 0))[0]
print theta

offset = [[0.5130161734617561]]
gradient = [ -1.08186669e-13  -1.64709490e-14]
[ 3.91520474  0.08564622]


In [12]:
train_data = data[:int(len(data)/2)]
test_data = data[int(len(data)/2):]

In [13]:
def feature(datum):
  if datum['beer/style'] == 'American IPA':
    feat = [1,1]
  else:
    feat = [1,0]
  return feat

X = [feature(d) for d in train_data]
y = [d['review/taste'] for d in train_data]
newtheta,residuals,rank,s = numpy.linalg.lstsq(X, y)

### Gradient descent ###
# Objective
def f(theta, X, y, lam):
  theta = numpy.matrix(theta).T
  X = numpy.matrix(X)
  y = numpy.matrix(y).T
  diff = X*theta - y
  diffSq = diff.T*diff
  diffSqReg = diffSq / len(X) + lam*(theta.T*theta)
  print "offset =", diffSqReg.flatten().tolist()
  return diffSqReg.flatten().tolist()[0]
# Derivative
def fprime(theta, X, y, lam):
  theta = numpy.matrix(theta).T
  X = numpy.matrix(X)
  y = numpy.matrix(y).T
  diff = X*theta - y
  res = 2*X.T*diff / len(X) + 2*lam*theta
  print "gradient =", numpy.array(res.flatten().tolist()[0])
  return numpy.array(res.flatten().tolist()[0])

theta = scipy.optimize.fmin_l_bfgs_b(f, [newtheta[0],newtheta[1]], fprime, args = (X, y, 0))[0]

offset = [[0.5581072865586575]]
gradient = [ -8.32005753e-14  -5.42968337e-15]


In [14]:
scipy.optimize.fmin_l_bfgs_b(f, [newtheta[0],newtheta[1]], fprime, args = (X, y, 0))

offset = [[0.5581072865586575]]
gradient = [ -8.32005753e-14  -5.42968337e-15]


(array([ 3.90435639,  0.05606027]),
 [0.5581072865586575],
 {'funcalls': 1,
  'grad': array([ -8.32005753e-14,  -5.42968337e-15]),
  'nit': 0,
  'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'warnflag': 0})

In [15]:
def feature(datum):
  if datum['beer/style'] == 'American IPA':
    feat = [1,1]
  else:
    feat = [1,0]
  return feat

X = [feature(d) for d in test_data]
y = [d['review/taste'] for d in test_data]
count = 0
sum_Diff_square = 0
for testData in test_data:
    sum_Diff_square += (y[count] - theta * numpy.matrix(X[count]).T) ** 2
    count += 1
test_MSE = sum_Diff_square / count
print test_MSE

X = [feature(d) for d in train_data]
y = [d['review/taste'] for d in train_data]
count = 0
sum_Diff_square = 0
for trainData in train_data:
    sum_Diff_square += (y[count] - theta * numpy.matrix(X[count]).T) ** 2
    count += 1
train_MSE = sum_Diff_square / count
print train_MSE

# MSE is the answer for problem 3

[[ 0.46841005]]
[[ 0.55810729]]


In [17]:
style_50more = {}
for style in style_num.keys():
    if style_num[style] >= 50:
        style_50more[style] = style_num[style]

print len(style_num)
print len(style_50more)

95
74


In [20]:
X = []
for single_data in train_data:
    X_tmp = [1]
    for style in style_50more:
        def feature(datum):
            if datum['beer/style'] == style:
                feat = [1]
            else:
                feat = [0]
            return feat
        X_tmp += feature(single_data)
    X.append(X_tmp)
y = [d['review/taste'] for d in train_data]
theta,residuals,rank,s = numpy.linalg.lstsq(X, y)
print len(theta)
print 'theta = ', theta
print residuals

75
theta =  [ 3.60681818  0.05895722 -0.50681818 -0.10681818 -0.62765152  0.19621212
  0.35359848  0.69564175  0.16385851 -0.18884943 -1.00681818  0.22651515
 -0.22045455 -0.45410882  0.02739474  0.76540404  0.3449362   0.38356643
 -0.10223103  0.8416167   0.48544372 -0.41285266 -1.23390152 -0.73802386
  0.01818182  0.60472028  0.26818182  0.09815076 -0.2937747  -0.1798951
  0.20984848 -0.65227273  0.11320382  0.10049889 -0.13106061 -0.58598485
  0.26761104  0.33596789  0.20312334  0.01699134  0.33580477  0.13786267
  0.1527972  -0.02450111 -0.4664673   0.46842454 -0.20681818  0.45638407
  0.03420746 -0.31931818  0.12193999  0.25681818 -0.27061129 -0.63806818
  0.12651515  0.44318182  0.71136364 -0.0058248  -0.11634199 -0.83277972
  0.65508658 -0.35681818 -0.17824675 -0.38622995  0.3763751  -0.74318182
  0.11433566  0.3305986   0.58195733 -0.09003966 -0.92019846  0.12395105
  0.26709486  0.39318182  0.14466111]
[ 9196.00692725]


In [23]:
### Gradient descent ###
# Objective
def f(theta, X, y, lam):
  theta = numpy.matrix(theta).T
  X = numpy.matrix(X)
  y = numpy.matrix(y).T
  diff = X*theta - y
  diffSq = diff.T*diff
  diffSqReg = diffSq / len(X) + lam*(theta.T*theta)
  print "offset =", diffSqReg.flatten().tolist()
  return diffSqReg.flatten().tolist()[0]
# Derivative
def fprime(theta, X, y, lam):
  theta = numpy.matrix(theta).T
  X = numpy.matrix(X)
  y = numpy.matrix(y).T
  diff = X*theta - y
  res = 2*X.T*diff / len(X) + 2*lam*theta
  print "gradient =", numpy.array(res.flatten().tolist()[0])
  return numpy.array(res.flatten().tolist()[0])

scipy.optimize.fmin_l_bfgs_b(f, theta, fprime, args = (X, y, 0))

offset = [[0.36784027709014366]]
gradient = [  1.20053301e-14  -3.99609235e-16  -1.05728759e-16  -3.19744231e-18
   1.84741111e-16  -7.23616722e-16  -1.20121513e-14   2.28027375e-14
   2.20268248e-18   6.36646291e-17   4.02167188e-17  -1.05444542e-16
  -1.49675827e-16  -6.92352842e-16   3.91366939e-16  -6.04245542e-16
  -1.58308922e-16   2.75406364e-16  -5.54223334e-17  -6.86668500e-16
   4.59522198e-15   5.22959454e-17   9.76214665e-16   2.40447662e-16
  -8.81072992e-18  -9.57811608e-17   4.53681537e-17   1.44382284e-16
   8.14281975e-17   3.16902060e-17  -1.32160949e-17  -5.73407988e-17
   3.25144356e-16  -2.30215846e-17   1.97246663e-16  -1.71667125e-16
  -3.54134500e-16  -2.19529284e-15   5.88187277e-16   2.58069122e-16
  -6.73239242e-17   4.06430445e-17   1.11768372e-16   2.19912977e-16
  -5.18696197e-18   7.61701813e-16  -5.36459765e-18   5.91171556e-17
   1.04591891e-16   1.50066626e-16   2.50963694e-16   4.01456646e-18
  -1.90709670e-16   3.63797881e-16   8.01492206e-17  -1.001

(array([ 3.60681818,  0.05895722, -0.50681818, -0.10681818, -0.62765152,
         0.19621212,  0.35359848,  0.69564175,  0.16385851, -0.18884943,
        -1.00681818,  0.22651515, -0.22045455, -0.45410882,  0.02739474,
         0.76540404,  0.3449362 ,  0.38356643, -0.10223103,  0.8416167 ,
         0.48544372, -0.41285266, -1.23390152, -0.73802386,  0.01818182,
         0.60472028,  0.26818182,  0.09815076, -0.2937747 , -0.1798951 ,
         0.20984848, -0.65227273,  0.11320382,  0.10049889, -0.13106061,
        -0.58598485,  0.26761104,  0.33596789,  0.20312334,  0.01699134,
         0.33580477,  0.13786267,  0.1527972 , -0.02450111, -0.4664673 ,
         0.46842454, -0.20681818,  0.45638407,  0.03420746, -0.31931818,
         0.12193999,  0.25681818, -0.27061129, -0.63806818,  0.12651515,
         0.44318182,  0.71136364, -0.0058248 , -0.11634199, -0.83277972,
         0.65508658, -0.35681818, -0.17824675, -0.38622995,  0.3763751 ,
        -0.74318182,  0.11433566,  0.3305986 ,  0.5

In [21]:
theta = scipy.optimize.fmin_l_bfgs_b(f, theta, fprime, args = (X, y, 0))[0]
X = []
for single_data in test_data:
    X_tmp = [1]
    for style in style_50more:
        def feature(datum):
            if datum['beer/style'] == style:
                feat = [1]
            else:
                feat = [0]
            return feat
        X_tmp += feature(single_data)
    X.append(X_tmp)
y = [d['review/taste'] for d in test_data]
count = 0
sum_Diff_square = 0
for testData in test_data:
    sum_Diff_square += (y[count] - theta * numpy.matrix(X[count]).T) ** 2
    count += 1
test_MSE = sum_Diff_square / count
print test_MSE

offset = [[0.36784027709014366]]
gradient = [  1.23145583e-14  -3.93640676e-16  -1.04307674e-16  -1.27897692e-18
   1.73656645e-16  -7.40385531e-16  -1.20689947e-14   2.28027375e-14
  -1.09423581e-17   5.74473802e-17   4.37694325e-17  -1.05444542e-16
  -1.41682222e-16  -6.64499566e-16   3.83693077e-16  -5.91455773e-16
  -1.50208734e-16   2.66169309e-16  -5.84421400e-17  -6.86668500e-16
   4.59749572e-15   6.16751095e-17   1.02772901e-15   2.53805865e-16
  -7.38964445e-18  -9.57811608e-17   4.68958206e-17   1.44382284e-16
   9.48929824e-17   2.91322522e-17  -2.60058641e-17  -5.60618219e-17
   3.80850906e-16  -1.53477231e-17   1.73798753e-16  -1.78488335e-16
  -3.56976670e-16  -2.04749995e-15   6.00266503e-16   2.44426701e-16
  -6.73239242e-17   3.21165317e-17   1.05941922e-16   1.92770244e-16
  -1.42108547e-18   7.61701813e-16  -4.83169060e-18  -6.42330633e-17
   9.35074240e-17   1.33013600e-16   2.33626452e-16   8.88178420e-19
  -1.72235559e-16   3.63797881e-16   6.77857770e-17  -1.037

In [3]:
# %load week1
import numpy
import urllib
import scipy.optimize
import random

def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)

print "Reading data..."
data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))
print "done"

def feature(datum):
  feat = [1]
  return feat

X = [feature(d) for d in data]
y = [d['review/overall'] for d in data]
theta,residuals,rank,s = numpy.linalg.lstsq(X, y)

### Convince ourselves that basic linear algebra operations yield the same answer ###

X = numpy.matrix(X)
y = numpy.matrix(y)
numpy.linalg.inv(X.T * X) * X.T * y.T

### Do older people rate beer more highly? ###

data2 = [d for d in data if d.has_key('user/ageInSeconds')]

def feature(datum):
  feat = [1]
  feat.append(datum['user/ageInSeconds'])
  return feat

X = [feature(d) for d in data2]
y = [d['review/overall'] for d in data2]
theta,residuals,rank,s = numpy.linalg.lstsq(X, y)

### How much do women prefer beer over men? ###

data2 = [d for d in data if d.has_key('user/gender')]

def feature(datum):
  feat = [1]
  if datum['user/gender'] == "Male":
    feat.append(0)
  else:
    feat.append(1)
  return feat

X = [feature(d) for d in data2]
y = [d['review/overall'] for d in data2]
theta,residuals,rank,s = numpy.linalg.lstsq(X, y)

### Gradient descent ###

# Objective
def f(theta, X, y, lam):
  theta = numpy.matrix(theta).T
  X = numpy.matrix(X)
  y = numpy.matrix(y).T
  diff = X*theta - y
  diffSq = diff.T*diff
  diffSqReg = diffSq / len(X) + lam*(theta.T*theta)
  print "offset =", diffSqReg.flatten().tolist()
  return diffSqReg.flatten().tolist()[0]

# Derivative
def fprime(theta, X, y, lam):
  theta = numpy.matrix(theta).T
  X = numpy.matrix(X)
  y = numpy.matrix(y).T
  diff = X*theta - y
  res = 2*X.T*diff / len(X) + 2*lam*theta
  print "gradient =", numpy.array(res.flatten().tolist()[0])
  return numpy.array(res.flatten().tolist()[0])

scipy.optimize.fmin_l_bfgs_b(f, [0,0], fprime, args = (X, y, 0.1))

### Random features ###

def feature(datum):
  return [random.random() for x in range(30)]

X = [feature(d) for d in data2]
y = [d['review/overall'] for d in data2]
theta,residuals,rank,s = numpy.linalg.lstsq(X, y)


Reading data...
done
offset = [[15.730542077145518]]
gradient = [-7.8098809  -0.12091359]
offset = [[9.019956530887699]]
gradient = [-5.60967714 -0.08716214]
offset = [[1.8678065242136093]]
gradient = [  1.53716340e-05  -9.92863150e-04]
offset = [[1.8678060995104089]]
gradient = [  1.39425607e-05  -8.89205217e-04]
offset = [[1.8678043781773557]]
gradient = [  2.44138043e-13   8.50014503e-17]
